In [5]:
import random
from base64 import b64decode
from json import loads
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans
import sklearn
from scipy.stats import mode
from sklearn.metrics import confusion_matrix
import seaborn as sns; sns.set()

In [6]:
def ler(x):

    digit = loads(x)
    array = np.fromstring(b64decode(digit["data"]),dtype=np.ubyte)
    array = array.astype(np.float64)
    return (digit["label"], array)

In [7]:
with open("digits.base64.json","r") as f:
    digits = map(ler, f.readlines())
digits=list(digits)

In [8]:
def display_digit(digit, labeled = True, title = ""):
    if labeled:
        digit = digit[1]
    image = digit
    plt.figure()
    fig = plt.imshow(image.reshape(28,28))
    fig.set_cmap('gray_r')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    if title != "":
        plt.title("Inferred label: " + str(title))

In [9]:
new_digits=[]
cont=0
while cont<len(digits):
    new_digits.append(digits[cont][1])
    cont+=1

In [10]:
digits_treino=digits[:len(digits)*5//6]
digits_val=digits[len(digits)*5//6:]
new_digits_treino=new_digits[:len(new_digits)*5//6]
new_digits_val=new_digits[len(new_digits)*5//6:]

In [11]:
r=[]
y=[]
cont=0
while cont<len(digits):
    r.append(digits[cont][0])
    cont+=1
y=[[[0] for i in np.arange(0,10)] for j in np.arange(0,60000)]
cont=0
while cont<len(digits):
    y[cont][r[cont]][0]=1
    cont+=1

Neural Network

In [89]:
n1=784
n2=30
n3=10
size=[n1,n2,n3]

In [90]:
def sig(x, deriv=False):
    
    if deriv:
        return sig(x)*(1-sig(x))
    
    return 1/(1+np.exp(-x))

In [91]:
def feedforward(i,w,b):
    z = []
    a = []
    a.append(i)
    cont=0
    while cont<2:
        z.append(np.add(np.dot(w[cont],a[cont]),b[cont]))
        var = sig(z[cont])
        a.append(var)
        cont+=1
    return a,z

In [92]:
def custox(a, cont):
    var = a[2]-y[cont]
    var = np.linalg.norm(var)
    cx = (var**2)/2
    return cx

In [93]:
def BP(a,z,cont):
    delta=[[],[]]
    #BP1
    var=a[2]-y[cont]
    var2=sig(z[1],deriv=True)
    delta[1]=np.multiply(var,var2)
    #BP2
    var=np.dot(w[1].transpose(),delta[1])
    var2=sig(z[0],deriv=True)
    delta[0]=np.multiply(var,var2)
    #BP3
    deriv_b=[[],[]]
    cont=0
    while cont<len(delta[0]):
        deriv_b[0].append(delta[0][cont])
        cont+=1
    cont=0
    while cont<len(delta[1]):
        deriv_b[1].append(delta[1][cont])
        cont+=1
    #BP4
    deriv_w = [0,0]
    matrix = [[0 for i in np.arange(0,n1)] for j in np.arange(0,n2)]
    cont=0
    while cont<len(delta[0]):
        cont2=0
        while cont2<len(a[0]):
            matrix[cont][cont2] = a[0][cont2][0]*delta[0][cont][0]
            #if cont == 0 and cont2 == 0:
                #print(a[0][cont2])
                #print(delta[0][cont])
            cont2+=1
        cont+=1
    deriv_w[0]=matrix
    
    matrix = [[0 for i in np.arange(0,n2)] for j in np.arange(0,n3)]
    cont=0
    while cont<len(delta[1]):
        cont2=0
        while cont2<len(a[1]):
            matrix[cont][cont2] = a[1][cont2][0]*delta[1][cont][0]
            cont2+=1
        cont+=1
    deriv_w[1]=matrix
    return deriv_w, deriv_b

In [94]:
def mini_batch(l,w,b):
    l_deriv_w0=[]
    l_deriv_w1=[]
    l_deriv_b0=[]
    l_deriv_b1=[]
    cont=0
    while cont<len(l):
        i=np.resize(new_digits[l[cont]],(784,1))
        a,z = feedforward(i,w,b)
        deriv_w, deriv_b = BP(a,z,l[cont])
        l_deriv_w0.append(deriv_w[0])
        l_deriv_w1.append(deriv_w[1])
        l_deriv_b0.append(deriv_b[0])
        l_deriv_b1.append(deriv_b[1])
        cont+=1
    return l_deriv_w0,l_deriv_w1, l_deriv_b0, l_deriv_b1
        
    

In [100]:
def epoch(eta,w,b):
    passo=0
    todos=np.arange(0,50000)
    np.random.shuffle(todos)
    while len(todos) != 0:
        l_deriv_w0,l_deriv_w1, l_deriv_b0, l_deriv_b1 = mini_batch(todos[:100],w,b)
        todos=todos[100:]
        mean_deriv_w0=l_deriv_w0[0]
        mean_deriv_w1=l_deriv_w1[0]
        mean_deriv_b0=l_deriv_b0[0]
        mean_deriv_b1=l_deriv_b1[0]
        cont=1
        while cont < len(l_deriv_w0):
            mean_deriv_w0 = np.add(mean_deriv_w0,l_deriv_w0[cont])
            cont+=1
        mean_deriv_w0 = np.multiply(eta/len(l_deriv_w0),mean_deriv_w0)
        
        cont=1
        while cont < len(l_deriv_w1):
            mean_deriv_w1 = np.add(mean_deriv_w1,l_deriv_w1[cont])
            cont+=1
        mean_deriv_w1 = np.multiply(eta/len(l_deriv_w1),mean_deriv_w1)
        
        cont=1
        while cont < len(l_deriv_b0):
            mean_deriv_b0 = np.add(mean_deriv_b0,l_deriv_b0[cont])
            cont+=1
        mean_deriv_b0 = np.multiply(eta/len(l_deriv_b0),mean_deriv_b0)
        
        cont=1
        while cont < len(l_deriv_b1):
            mean_deriv_b1 = np.add(mean_deriv_b1,l_deriv_b1[cont])
            cont+=1
        mean_deriv_b1 = np.multiply(eta/len(l_deriv_b1),mean_deriv_b1)
        w[0] = np.subtract(w[0],mean_deriv_w0)
        w[1] = np.subtract(w[1],mean_deriv_w1)
        b[0] = np.subtract(b[0],mean_deriv_b0)
        b[1] = np.subtract(b[1],mean_deriv_b1)
        passo+=100
        if passo%5000==0:
            print(passo)
    return w,b

In [101]:
b
b = [np.random.randn(y, 1) for y in size[1:]]
w = [np.random.randn(y, x) for x, y in zip(size[:-1], size[1:])]
w,b=epoch(3,w,b)

C:\Users\User-pc\Anaconda\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in exp
  


5000
10000
15000
20000
25000
30000
35000
40000
45000
50000


([array([   0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
            0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,

In [114]:
i=new_digits[0]
a,z=feedforward(i,w,b)
len(a[1][0])

C:\Users\User-pc\Anaconda\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: overflow encountered in exp
  


30

In [104]:
a[2]

array([[  3.28182316e-01,   4.11751792e-01,   3.28182316e-01,
          3.28182316e-01,   3.28182316e-01,   3.28182316e-01,
          4.11751792e-01,   3.28182316e-01,   4.11751792e-01,
          3.28182316e-01,   3.28182316e-01,   3.28182316e-01,
          3.28182316e-01,   4.11751792e-01,   3.28182316e-01,
          4.11751792e-01,   3.28182316e-01,   4.11751792e-01,
          3.28182316e-01,   4.11751792e-01,   4.11751792e-01,
          4.11751792e-01,   3.28182316e-01,   4.11751792e-01,
          4.11751792e-01,   4.11751792e-01,   3.28182316e-01,
          4.11751792e-01,   3.28182316e-01,   3.28182316e-01],
       [  3.43415610e-01,   1.55810949e-04,   3.43415610e-01,
          3.43415610e-01,   3.43415610e-01,   3.43415610e-01,
          1.55810949e-04,   3.43415610e-01,   1.55810949e-04,
          3.43415610e-01,   3.43415610e-01,   3.43415610e-01,
          3.43415610e-01,   1.55810949e-04,   3.43415610e-01,
          1.55810949e-04,   3.43415610e-01,   1.55810949e-04,
       

In [42]:
[1,2,3,4,5,6,7,8,9][:4]
a=[1,2,3,4,5,6,7,8,9][4:]
a[4:]
eta=3

[9]

In [74]:
aa=[[1 for i in np.arange(0,10)] for i in np.arange(0,10)]
np.matrix(aa)
ab=[[1 for i in np.arange(0,10)] for i in np.arange(0,10)]
np.matrix(ab)
#np.add(aa,ab)
np.subtract(aa,ab)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [99]:
5%3

2